### Henry Analysis

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set_style("whitegrid")

In [22]:
from data_loader import data_load_and_split
from data_loader import data_merge

ImportError: cannot import name 'data_merge' from 'data_loader' (/Users/henrycladouhos/Documents/GitHub/Erdos_Trees/machine_learning/data_loader.py)

In [15]:
big_firstMeas_train, big_secondMeas_train, big_firstMeas_test, big_secondMeas_test, trimmed_firstMeas_train, trimmed_secondMeas_train, trimmed_firstMeas_test, trimmed_secondMeas_test = data_load_and_split()


In [ ]:
combined_train =data_merge(trimmed_firstMeas_train, trimmed_secondMeas_train)